In [ ]:
from workflows import DYStudiesProcessor

import json
from coffea.processor import (
    run_uproot_job,
    iterative_executor, 
    futures_executor,
    NanoAODSchema
)

with open("dummy_samples.json") as f:
    samples = json.load(f)

import json
import os.path as osp
metaconditions_file = "metaconditions/Era2017_legacy_v1.json"
with open(metaconditions_file) as f:
    metaconditions = json.load(f)    

dystudies = DYStudiesProcessor(metaconditions, False, True, './outputs/')

histos = run_uproot_job(
    samples,
    'Events',
    dystudies,
    executor=futures_executor,
    executor_args={"schema": NanoAODSchema,"workers": 4},
    )




In [ ]:
x = ak.ones_like(ak.flatten(events.Photon.pt))

In [3]:
import pandas as pd
import pyarrow as pa
import awkward as ak

x = pd.read_parquet('./outputs/')

#y = ak.from_arrow(pa.Table.from_pandas(x))

In [4]:
x

,lead_charge,lead_cleanmask,lead_cutBased,lead_cutBased_Fall17V1Bitmap,lead_eCorr,lead_electronIdx,lead_electronIdxG,lead_electronVeto,lead_energyErr,lead_eta,...,pt,eta,phi,mass,DummyTagger2_10,DummyTagger1_20,best_tag,event,lumi,run
0,0,1,3,7,1.007115,0,262,True,0.492188,-0.094513,...,6.484444,2.477199,-1.648825,90.491798,10,0,10,50899923,203600,1
1,0,1,2,7,0.996568,0,732,False,4.562500,-2.109863,...,5.304607,-4.871169,1.360900,91.277992,10,0,10,225504595,902019,1
2,0,1,3,7,1.007751,0,748,False,0.773438,-0.357788,...,30.430706,-1.303718,2.839696,91.794151,10,0,10,225504665,902019,1
3,0,1,3,7,1.016430,0,753,False,2.031250,-1.001221,...,39.983696,-1.645426,-0.794251,91.972687,10,0,10,225504676,902019,1
4,0,1,3,7,1.005413,1,757,False,0.523438,-0.630127,...,3.725598,1.051991,-1.337834,91.603516,10,0,10,225504715,902019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1930,0,1,3,7,0.998379,0,66895,False,0.953125,-0.834839,...,2.390588,-3.400738,-1.279562,93.078018,10,0,10,88749602,354999,1
1931,0,1,1,7,0.999010,0,66987,False,0.968750,0.622437,...,30.884182,0.498902,0.457023,90.186493,10,0,10,88749984,355000,1
1932,0,1,1,7,1.017706,0,67024,False,4.875000,2.218262,...,5.874158,4.508657,-2.799278,95.356773,10,0,10,88750109,355001,1
1933,0,1,2,3,0.997854,0,67073,False,0.742188,-0.662842,...,9.362647,-1.871684,-0.667799,90.999992,10,0,10,88750300,355002,1


In [ ]:
# di-photon MVA
import xgboost as xg

min_diphoton_mass = 100
max_diphoton_mass = 180

model = xg.Booster()
model.load_model('aux-data/altDiphoModel_coffea.model')

# get the number of diphotons per row
# and save for re-wrapping xgb outputs
counts = ak.num(diphotons, axis=1)

# extract diphoton vars into flat lists
dipho_leadIDMVA = ak.flatten(diphotons["0"].mvaID)
dipho_subleadIDMVA = ak.flatten(diphotons["1"].mvaID)
dipho_lead_ptoM = ak.flatten(diphotons["0"].pt / diphotons.mass)
dipho_sublead_ptoM = ak.flatten(diphotons["1"].pt / diphotons.mass)
dipho_lead_eta = ak.flatten(diphotons["0"].eta)
dipho_sublead_eta = ak.flatten(diphotons["1"].eta)

diphoVars  = ['dipho_leadIDMVA', 'dipho_subleadIDMVA', 'dipho_lead_ptoM', 
              'dipho_sublead_ptoM', 'dipho_leadEta', 'dipho_subleadEta', 
              'CosPhi', 'vtxprob', 'sigmarv', 'sigmawv']
allVars = diphoVars + ["dipho_mass"]

f = uproot.open('data/ggH_powheg_UL_2017.root')
tree = f['vbfTagDumper/trees/ggh_125_13TeV_GeneralDipho']
arrays = tree.arrays(allVars, how=dict)

mask = (  (arrays["dipho_mass"]> min_diphoton_mass) & (arrays["dipho_mass"]< max_diphoton_mass) 
        & (arrays["dipho_leadIDMVA"]>-0.9) & (arrays["dipho_subleadIDMVA"]>-0.9) 
        & (arrays["dipho_lead_ptoM"]>0.333) & (arrays["dipho_sublead_ptoM"]>0.25))

x = np.column_stack((ak.to_numpy(arrays[var][mask]) for var in diphoVars))[:100]

print(x.shape)

diphoMatrix = xg.DMatrix(x, feature_names=diphoVars)


y = model.predict(diphoMatrix)

In [ ]:
import uproot
import xgboost
import numpy as np
import awkward as ak

diphoVars  = ['dipho_leadIDMVA', 'dipho_subleadIDMVA', 'dipho_lead_ptoM', 
              'dipho_sublead_ptoM', 'dipho_leadEta', 'dipho_subleadEta', 
              'CosPhi', 'vtxprob', 'sigmarv', 'sigmawv']
allVars = diphoVars + ["dipho_mass"]

f = uproot.open('../data/ggH_powheg_UL_2017.root')
tree = f['vbfTagDumper/trees/ggh_125_13TeV_GeneralDipho']
arrays = tree.arrays(allVars, how=dict)

mask = (  (arrays["dipho_mass"]>100.) & (arrays["dipho_mass"]<180.) 
        & (arrays["dipho_leadIDMVA"]>-0.9) & (arrays["dipho_subleadIDMVA"]>-0.9) 
        & (arrays["dipho_lead_ptoM"]>0.333) & (arrays["dipho_sublead_ptoM"]>0.25))

x = np.column_stack((ak.to_numpy(arrays[var][mask]) for var in diphoVars))[:100]

print(x.shape)

diphoMatrix = xgboost.DMatrix(x, feature_names=diphoVars)

model = xgboost.Booster()
model.load_model('aux-data/altDiphoModel_coffea.model')
y = model.predict(diphoMatrix)

print(x)
print(y)

In [ ]:
events.behavior["__events_factory__"]._partition_key.replace('/', '_') + '.parquet'

In [ ]:
events.metadata

In [ ]:
metaconditions

In [ ]:
from workflows import taggers

In [ ]:
dir(taggers)

In [ ]:
import awkward as ak

x = ak.Array([[1.0], [2.0, 3.0], [4.0, 5.0, 6.0]])



In [ ]:
counts = ak.num(x, axis=1)
ak.unflatten([0, 0] * ak.flatten(x)[:, None], counts)

In [ ]:
import numpy as np
x = np.array([0,1,2])
y = np.array([3,4,5])

np.stack((x,y), axis=1)

In [ ]:
ak.Array([1,2,3,4])[ak.min(ak.Array([[1,-1], []]), axis=1)]

In [ ]:
str(['abc', 1, 5.0])

In [ ]:
eval("['abc', 1, 5.0]")

In [ ]:
import sys
sys.maxsize